In [46]:
import itertools
import numpy as np
import pandas as pd
# for data scaling and splitting
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
# for neural net
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# for evaluation
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

In [47]:
data = pd.read_csv("data/combined_expression.csv")
data.head()
data.shape

(642, 16383)

In [48]:
selected_genes = pd.read_csv('cleaned/boruta-99-25-0.01.csv')
selected_genes = selected_genes.values.tolist()
selected_genes = list(itertools.chain(*selected_genes))

In [49]:
# retrieving proper columns
X = data.loc[:, selected_genes]
y = data['classification'].values

# scaling the data
scalar = MinMaxScaler()
x_scaled = scalar.fit_transform(X)

# splitting data (20% test, 80% train)
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=0)

# Gridsearch for Input and Output Layer (no hidden layers)

## Optimizing Epochs and Batches

In [57]:
def create_model(optimizer='rmsprop',init='glorot_uniform'):
    model = Sequential()
    # adding layers and adding droplayers to avoid overfitting
    hidden_layers = len(selected_genes)
    model.add(Dense(hidden_layers, activation='relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense((hidden_layers*0.5), activation='relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense((hidden_layers*0.25), activation='relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense((hidden_layers*0.25), activation='relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(1, activation='sigmoid'))
    # compiling
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [58]:
model = KerasClassifier(build_fn=create_model)
epochs = [50, 75, 100, 150]
batches = [16, 32, 64, 128]
optimizers = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
init = ['glorot_uniform', 'normal', 'uniform']
param_grid = dict(epochs=epochs, batch_size=batches,optimizer=optimizers,init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

Fitting 3 folds for each of 336 candidates, totalling 1008 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 16.6min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 26.2min
[Parallel(n_jobs=-1)]: Done 1008 out of 1008 | elapsed: 31.3min finished


Epoch 1/100
513/513 [==============================] - 0s 951us/sample - loss: 0.6607 - acc: 0.6121
Epoch 2/100
513/513 [==============================] - 0s 89us/sample - loss: 0.6651 - acc: 0.6472
Epoch 3/100
513/513 [==============================] - 0s 101us/sample - loss: 0.6380 - acc: 0.6530
Epoch 4/100
513/513 [==============================] - 0s 93us/sample - loss: 0.6318 - acc: 0.6413
Epoch 5/100
513/513 [==============================] - 0s 91us/sample - loss: 0.6265 - acc: 0.6569
Epoch 6/100
513/513 [==============================] - 0s 97us/sample - loss: 0.6364 - acc: 0.6472
Epoch 7/100
513/513 [==============================] - 0s 95us/sample - loss: 0.6344 - acc: 0.6452
Epoch 8/100
513/513 [==============================] - 0s 231us/sample - loss: 0.6254 - acc: 0.6608
Epoch 9/100
513/513 [==============================] - 0s 109us/sample - loss: 0.6229 - acc: 0.6589
Epoch 10/100
513/513 [==============================] - 0s 109us/sample - loss: 0.6246 - acc: 0.6569
Epoc

513/513 [==============================] - 0s 218us/sample - loss: 0.5677 - acc: 0.6979
Epoch 83/100
513/513 [==============================] - 0s 99us/sample - loss: 0.5770 - acc: 0.7232
Epoch 84/100
513/513 [==============================] - 0s 90us/sample - loss: 0.5724 - acc: 0.7096
Epoch 85/100
513/513 [==============================] - 0s 95us/sample - loss: 0.5645 - acc: 0.7057
Epoch 86/100
513/513 [==============================] - 0s 91us/sample - loss: 0.5750 - acc: 0.7310
Epoch 87/100
513/513 [==============================] - 0s 92us/sample - loss: 0.5500 - acc: 0.7193
Epoch 88/100
513/513 [==============================] - 0s 93us/sample - loss: 0.5717 - acc: 0.7076
Epoch 89/100
513/513 [==============================] - 0s 85us/sample - loss: 0.5477 - acc: 0.7096
Epoch 90/100
513/513 [==============================] - 0s 109us/sample - loss: 0.5697 - acc: 0.7193
Epoch 91/100
513/513 [==============================] - 0s 91us/sample - loss: 0.5605 - acc: 0.7135
Epoch 92/10

In [59]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.754386 using {'batch_size': 64, 'epochs': 100, 'init': 'normal', 'optimizer': 'Adagrad'}


In [41]:
grid_result.cv_results_



{'mean_fit_time': array([ 7.22039588,  9.56947915,  8.48573939,  9.27067685,  8.95816739,
         8.88102937, 14.03832571,  7.63566136, 10.87969049,  9.99364114,
        10.72668576,  9.70933819,  9.85021536, 15.11698103,  8.59357548,
        11.95666877, 10.56142751, 10.84465106,  9.85390568, 10.20533554,
        14.22944824, 10.99437936, 15.08111501, 13.09001851, 15.21255986,
        13.07044125, 12.89606134, 19.08121816, 11.24616599, 14.71461336,
        11.30761361, 12.28670128, 11.74078059, 11.31368788, 17.73999445,
        10.78163417, 13.59046125, 12.35867484, 13.70239774, 12.40315477,
        12.52348463, 18.69288786, 13.24676792, 17.92378426, 15.24838185,
        17.705592  , 15.63943529, 15.32981396, 24.04683884, 13.98553594,
        18.01362753, 15.96535977, 16.85418582, 15.94847647, 16.37439966,
        25.11128616, 12.991738  , 15.1353151 , 13.29384446, 15.43697381,
        14.84020011, 14.71931307, 23.02711749, 18.31621861, 24.135686  ,
        22.02819777, 25.05460095, 

In [65]:
y_pred = grid.predict(X_test)

## Tuning the Training Optimization Algorithm

# Testing the Model

In [60]:
model = KerasClassifier(build_fn=create_model, epochs=grid_result.best_params_['epochs'], batch_size=grid_result.best_params_['batch_size'],optimizer=grid_result.best_params_['optimizer'],init=grid_result.best_params_['init'])
kfold = KFold(n_splits=3, shuffle=True)
results = cross_val_score(model, X_train, y_train, cv=kfold)
print("Baseline Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100
342/342 [==============================] - 1s 1ms/sample - loss: 0.6770 - acc: 0.5760
Epoch 2/100
342/342 [==============================] - 0s 120us/sample - loss: 0.6228 - acc: 0.6754
Epoch 3/100
342/342 [==============================] - 0s 139us/sample - loss: 0.6106 - acc: 0.6725
Epoch 4/100
342/342 [==============================] - 0s 117us/sample - loss: 0.6288 - acc: 0.6725
Epoch 5/100
342/342 [==============================] - 0s 166us/sample - loss: 0.6389 - acc: 0.6696
Epoch 6/100
342/342 [==============================] - 0s 128us/sample - loss: 0.6356 - acc: 0.6754
Epoch 7/100
342/342 [==============================] - 0s 125us/sample - loss: 0.6214 - acc: 0.6725
Epoch 8/100
342/342 [==============================] - 0s 155us/sample - loss: 0.6091 - acc: 0.6667
Epoch 9/100
342/342 [==============================] - 0s 155us/sample - loss: 0.6129 - acc: 0.6813
Epoch 10/100
342/342 [==============================] - 0s 137us/sample - loss: 0.6175 - acc: 0.6842
E

342/342 [==============================] - 0s 160us/sample - loss: 0.5199 - acc: 0.7427
Epoch 82/100
342/342 [==============================] - 0s 152us/sample - loss: 0.5133 - acc: 0.7251
Epoch 83/100
342/342 [==============================] - 0s 122us/sample - loss: 0.4945 - acc: 0.7632
Epoch 84/100
342/342 [==============================] - 0s 102us/sample - loss: 0.5166 - acc: 0.7544
Epoch 85/100
342/342 [==============================] - 0s 97us/sample - loss: 0.5180 - acc: 0.7544
Epoch 86/100
342/342 [==============================] - 0s 102us/sample - loss: 0.5020 - acc: 0.7661
Epoch 87/100
342/342 [==============================] - 0s 297us/sample - loss: 0.4866 - acc: 0.7573
Epoch 88/100
342/342 [==============================] - 0s 120us/sample - loss: 0.4876 - acc: 0.7719
Epoch 89/100
342/342 [==============================] - 0s 111us/sample - loss: 0.5192 - acc: 0.7661
Epoch 90/100
342/342 [==============================] - 0s 105us/sample - loss: 0.4998 - acc: 0.7339
Epoc

342/342 [==============================] - 0s 120us/sample - loss: 0.5189 - acc: 0.7456
Epoch 62/100
342/342 [==============================] - 0s 117us/sample - loss: 0.5264 - acc: 0.7602
Epoch 63/100
342/342 [==============================] - 0s 108us/sample - loss: 0.5057 - acc: 0.7368
Epoch 64/100
342/342 [==============================] - 0s 104us/sample - loss: 0.5369 - acc: 0.7339
Epoch 65/100
342/342 [==============================] - 0s 104us/sample - loss: 0.5197 - acc: 0.7661
Epoch 66/100
342/342 [==============================] - 0s 125us/sample - loss: 0.5118 - acc: 0.7281
Epoch 67/100
342/342 [==============================] - 0s 122us/sample - loss: 0.5477 - acc: 0.7456
Epoch 68/100
342/342 [==============================] - 0s 114us/sample - loss: 0.5274 - acc: 0.7485
Epoch 69/100
342/342 [==============================] - 0s 103us/sample - loss: 0.5407 - acc: 0.7281
Epoch 70/100
342/342 [==============================] - 0s 111us/sample - loss: 0.5092 - acc: 0.7749
Epo

342/342 [==============================] - 0s 116us/sample - loss: 0.5796 - acc: 0.7105
Epoch 42/100
342/342 [==============================] - 0s 113us/sample - loss: 0.5687 - acc: 0.7368
Epoch 43/100
342/342 [==============================] - 0s 102us/sample - loss: 0.5637 - acc: 0.6988
Epoch 44/100
342/342 [==============================] - 0s 99us/sample - loss: 0.5642 - acc: 0.7193
Epoch 45/100
342/342 [==============================] - 0s 108us/sample - loss: 0.5777 - acc: 0.7047
Epoch 46/100
342/342 [==============================] - 0s 315us/sample - loss: 0.5557 - acc: 0.7398
Epoch 47/100
342/342 [==============================] - 0s 105us/sample - loss: 0.5823 - acc: 0.6988
Epoch 48/100
342/342 [==============================] - 0s 102us/sample - loss: 0.5816 - acc: 0.6901
Epoch 49/100
342/342 [==============================] - 0s 119us/sample - loss: 0.5707 - acc: 0.6871
Epoch 50/100
342/342 [==============================] - 0s 108us/sample - loss: 0.5876 - acc: 0.7222
Epoc

In [61]:
model.fit(X_train, y_train)
test_predictions = model.predict(X_test)

Epoch 1/100
513/513 [==============================] - 1s 1ms/sample - loss: 0.6717 - acc: 0.5965
Epoch 2/100
513/513 [==============================] - 0s 107us/sample - loss: 0.6555 - acc: 0.6394
Epoch 3/100
513/513 [==============================] - 0s 103us/sample - loss: 0.6555 - acc: 0.6316
Epoch 4/100
513/513 [==============================] - 0s 113us/sample - loss: 0.6474 - acc: 0.6706
Epoch 5/100
513/513 [==============================] - 0s 222us/sample - loss: 0.6378 - acc: 0.6647
Epoch 6/100
513/513 [==============================] - 0s 108us/sample - loss: 0.6299 - acc: 0.6491
Epoch 7/100
513/513 [==============================] - 0s 113us/sample - loss: 0.6290 - acc: 0.6550
Epoch 8/100
513/513 [==============================] - 0s 94us/sample - loss: 0.6265 - acc: 0.6569
Epoch 9/100
513/513 [==============================] - 0s 93us/sample - loss: 0.6313 - acc: 0.6589
Epoch 10/100
513/513 [==============================] - 0s 99us/sample - loss: 0.6207 - acc: 0.6667
Epoc

513/513 [==============================] - 0s 91us/sample - loss: 0.5636 - acc: 0.7115
Epoch 83/100
513/513 [==============================] - 0s 99us/sample - loss: 0.5595 - acc: 0.7173
Epoch 84/100
513/513 [==============================] - 0s 91us/sample - loss: 0.5902 - acc: 0.6901
Epoch 85/100
513/513 [==============================] - 0s 247us/sample - loss: 0.5678 - acc: 0.7154
Epoch 86/100
513/513 [==============================] - 0s 94us/sample - loss: 0.5776 - acc: 0.7076
Epoch 87/100
513/513 [==============================] - 0s 91us/sample - loss: 0.5717 - acc: 0.7349
Epoch 88/100
513/513 [==============================] - 0s 95us/sample - loss: 0.5768 - acc: 0.6842
Epoch 89/100
513/513 [==============================] - 0s 100us/sample - loss: 0.5732 - acc: 0.7154
Epoch 90/100
513/513 [==============================] - 0s 93us/sample - loss: 0.5531 - acc: 0.7310
Epoch 91/100
513/513 [==============================] - 0s 89us/sample - loss: 0.5692 - acc: 0.7076
Epoch 92/10

In [66]:
print(classification_report(y_test, test_predictions))
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.91      0.77        75
           1       0.75      0.39      0.51        54

    accuracy                           0.69       129
   macro avg       0.71      0.65      0.64       129
weighted avg       0.71      0.69      0.66       129

              precision    recall  f1-score   support

           0       0.65      0.91      0.76        75
           1       0.72      0.33      0.46        54

    accuracy                           0.67       129
   macro avg       0.69      0.62      0.61       129
weighted avg       0.68      0.67      0.63       129



In [63]:
print(confusion_matrix(y_test,test_predictions))

[[68  7]
 [33 21]]


In [40]:
model.model.save('model_1.h5')